#  自动生成音乐





In [20]:
import mido
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import os

##  读取文件
```
timidity input.mid -Ow -o - | ffmpeg -i - -acodec libmp3lame -ab 256k out.mp3
```


In [21]:
mid = MidiFile('assets/sample.mid') 
output = 'output'

if not os.path.exists(output):
    os.mkdir(output)

notes = []

## 将声音文件解析成序列

In [3]:
notes = []
for msg in mid:
    if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
        data = msg.bytes()
        notes.append(data[1])

In [5]:
print('notes length: ', len(notes))


notes length:  13100


##  对数据进行归一化处理

In [7]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(np.array(notes).reshape(-1,1))
notes = list(scaler.transform(np.array(notes).reshape(-1,1)))

##  划分数据

In [8]:
# LSTM layers requires that data must have a certain shape
# create list of lists fist
notes = [list(note) for note in notes]

# subsample data for training and prediction
X = []
y = []
# number of notes in a batch
n_prev = 30
for i in range(len(notes)-n_prev):
    X.append(notes[i:i+n_prev])
    y.append(notes[i+n_prev])
# save a seed to do prediction later
X_test = X[-300:]
X = X[:-300]
y = y[:-300]

In [17]:
print("shape [{}, {}] ".format(len(X), len(X[0])))
for i in range(5):
    print("[{} ... {}]  ==> {:.8f}\t".format(X[i][0], X[i][-2:] , y[i][0]))


shape [12770, 30] 
[[0.33333333333333326] ... [[0.33333333333333326], [0.6666666666666667]]]  ==> 0.19444444	
[[0.6666666666666667] ... [[0.6666666666666667], [0.19444444444444442]]]  ==> 0.52777778	
[[0.3055555555555556] ... [[0.19444444444444442], [0.5277777777777777]]]  ==> 0.33333333	
[[0.6388888888888888] ... [[0.5277777777777777], [0.33333333333333326]]]  ==> 0.66666667	
[[0.25] ... [[0.33333333333333326], [0.6666666666666667]]]  ==> 0.30555556	


## 定义网络 

Made sequential model with several layers, use LSTM as it time dependent data

I also whant to save checkpoints

In [18]:
model = Sequential()
model.add(LSTM(256, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('linear'))
optimizer = Adam(lr=0.001)
model.compile(loss='mse', optimizer=optimizer)
filepath="./output/checkpoint_model_{epoch:02d}.hdf5"
model_save_callback = ModelCheckpoint(filepath, monitor='val_acc', 
                                      verbose=1, save_best_only=True, 
                                      mode='auto', period=5)




Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



##  开始训练
It might take a while, I was waiting for 1 hour with just 5 epoch

In [ ]:
epoch = 10
batch_size = 32
model.fit(np.array(X), np.array(y), batch_size, epoch, verbose=1, callbacks=[model_save_callback])

## 进行预测，生成新音乐

In [29]:
prediction = model.predict(np.array(X_test))
# print(prediction[0:10])
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]

# print(X_test[0])
# print(len(prediction))

[[0.63144195]
 [0.7169561 ]
 [0.8348223 ]]
[[0.5277777777777777], [0.0], [0.19444444444444442], [0.33333333333333326], [0.38888888888888884], [0.4444444444444444], [0.33333333333333326], [0.5277777777777777], [0.6666666666666667], [0.13888888888888884], [0.33333333333333326], [0.4722222222222221], [0.5277777777777777], [0.5833333333333333], [0.4722222222222221], [0.6666666666666667], [0.8055555555555556], [0.19444444444444442], [0.38888888888888884], [0.5277777777777777], [0.5833333333333333], [0.6388888888888888], [0.5277777777777777], [0.7222222222222221], [0.8611111111111112], [0.05555555555555558], [0.13888888888888884], [0.3055555555555556], [0.38888888888888884], [0.5277777777777777]]
3


## 保存音乐文件

In [10]:
mid = MidiFile()
track = MidiTrack()
t = 0
for note in prediction:
    # 147 means note_on
    # 67 is velosity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)
mid.tracks.append(track)
mid.save('LSTM_music.mid')

#### Just listen to it. The result is surreal!